# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [35]:
# import libraries
import numpy as np
import pandas as pd
import pickle

from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

In [11]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)  
X = df[['message', 'genre']]
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [15]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. The [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) is used for predicting multiple target variables.

In [16]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 20)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X.iloc[:,0], Y)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test the model
The f1 score, precision and recall for each output category of the dataset is shown by iterating through the columns and calling sklearn's `classification_report` on each.

Due to the class imbalance. The macro avgrage f1 score is used as the score and averaged over all the classes.

In [18]:
# predict on test data
Y_pred = pipeline.predict(X_test)

In [15]:
# display results
for i, col in enumerate(Y_test.columns):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:,i]))

related
              precision    recall  f1-score   support

           0       0.66      0.31      0.42      1500
           1       0.82      0.95      0.88      5054

    accuracy                           0.81      6554
   macro avg       0.74      0.63      0.65      6554
weighted avg       0.79      0.81      0.78      6554

request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5427
           1       0.84      0.40      0.54      1127

    accuracy                           0.88      6554
   macro avg       0.86      0.69      0.74      6554
weighted avg       0.88      0.88      0.87      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      655

/Users/jeremydai/anaconda/envs/p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6241
           1       0.80      0.09      0.16       313

    accuracy                           0.96      6554
   macro avg       0.88      0.54      0.57      6554
weighted avg       0.95      0.96      0.94      6554

electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6412
           1       1.00      0.01      0.01       142

    accuracy                           0.98      6554
   macro avg       0.99      0.50      0.50      6554
weighted avg       0.98      0.98      0.97      6554

tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6508
           1       0.00      0.00      0.00        46

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

h

In [32]:
# macro avgrage f1 score is used as the score
# the score on the given test data and labels.
def score(Y_test, Y_pred):
    f1 = []
    for i, col in enumerate(Y_test.columns):
        results = classification_report(Y_test[col], Y_pred[:,i], output_dict = True)
        f1.append(results['macro avg']['f1-score'])
    return np.mean(f1)

score(Y_test, Y_pred)

0.5981838953582067

### 6. Improve the model
Use grid search to find better parameters. Since this project focuses on building pipelines, we will not spend too much time optimizing the model. 

In [17]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
   # 'vect__max_df': (0.5, 0.75),
   # 'vect__max_features': (None, 5000),
   # 'tfidf__use_idf': (True, False),
   # 'clf__estimator__n_estimators': [50, 100],
   # 'clf__estimator__min_samples_split': [2, 3]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv = 3, scoring='f1_marco')


cv.fit(X_train, Y_train)

/Users/jeremydai/anaconda/envs/p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jeremydai/anaconda/envs/p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/jeremydai/anaconda/envs/p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jeremydai/anaconda/envs/p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

In [19]:
cv.best_params_
# cv.best_estimator_

{'vect__ngram_range': (1, 2)}

### 7. Test the model
Show the accuracy, precision, and recall of the tuned model.  

In [33]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, ngram_range = (1, 2))),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 20)))
    ])

In [34]:
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)
score(Y_test, Y_pred)

/Users/jeremydai/anaconda/envs/p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.6022067345213669

### 8. Some other ideas to improve the model further. 
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export the model as a pickle file

In [23]:
pickle.dump(cv, open("model.pkl", 'wb'))

### 10. Use this notebook to complete `train.py`

In [44]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, ngram_range = (1, 2))),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 20)))
    ])
    return pipeline

In [45]:
model = build_model()


In [47]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
    X = df[['message']]
    Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

['related',
 'aid_related',
 'weather_related',
 'direct_report',
 'request',
 'other_aid',
 'food',
 'earthquake',
 'storm',
 'shelter',
 'floods',
 'medical_help',
 'infrastructure_related',
 'water',
 'other_weather',
 'buildings',
 'medical_products',
 'transport',
 'death',
 'other_infrastructure',
 'refugees',
 'military',
 'search_and_rescue',
 'money',
 'electricity',
 'cold',
 'security',
 'clothing',
 'aid_centers',
 'missing_people',
 'hospitals',
 'fire',
 'tools',
 'shops',
 'offer',
 'child_alone']

In [66]:
`dd

[20093,
 10860,
 7297,
 5075,
 4474,
 3446,
 2923,
 2455,
 2443,
 2314,
 2155,
 2084,
 1705,
 1672,
 1376,
 1333,
 1313,
 1201,
 1194,
 1151,
 875,
 860,
 724,
 604,
 532,
 530,
 471,
 405,
 309,
 298,
 283,
 282,
 159,
 120,
 118,
 0]

In [54]:
Y_train.shape

(20972, 36)

In [55]:
model 

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  